In [1]:
from diffusers import UNet2DModel, DDPMScheduler
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
import torch

import os
import re


In [2]:
batch_size = 64
learning_rate = 1e-4
num_epochs = 100
image_size = 64
data_path = "data/bedroom"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [3]:
def find_folder_with_max_number(base_path: str, type_name: str) -> int:
    max_number = -1

    regex_ = f"{type_name}_" + r"(\d+)"
    pattern = re.compile(regex_)
    
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)

        if os.path.isdir(item_path):
            match = pattern.match(item)
            if match:
                number = int(match.group(1))
                if number > max_number:
                    max_number = number

    return max_number


In [4]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

dataset = datasets.ImageFolder(data_path, transform=transform)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [7]:
last_epoch = find_folder_with_max_number("model_paths", "all_data")
print("Last epoch:", last_epoch)

if last_epoch == -1:
    path = "google/ddpm-cifar10-32"
else:
    path = f"model_paths/all_data_{last_epoch}"


Last epoch: 1


In [8]:
model = UNet2DModel.from_pretrained(path)
model.to(device)

optimizer = AdamW(model.parameters(), lr=learning_rate)

scheduler = DDPMScheduler(
    num_train_timesteps=1000,
    beta_start=0.0001,
    beta_end=0.02,
    beta_schedule="linear"
)


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [6]:
for epoch in range(last_epoch, last_epoch+10):
    model.train()

    for data in tqdm(dataloader):
        inputs, _ = data
        inputs = inputs.to(device)

        noise = torch.randn_like(inputs)
        timesteps = torch.randint(
            0,
            scheduler.num_train_timesteps,
            (inputs.shape[0],),
            device=device
            ).long()

        noisy_inputs = scheduler.add_noise(inputs, noise, timesteps)

        noise_pred = model(noisy_inputs, timesteps).sample
        loss = torch.nn.functional.mse_loss(noise_pred, noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.save_pretrained(f"{epoch}_path_to_save_model")
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


  0%|          | 0/19 [00:00<?, ?it/s]c:\Users\Studia\anaconda3\lib\site-packages\diffusers\configuration_utils.py:140: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
  0%|          | 0/19 [00:21<?, ?it/s]


KeyboardInterrupt: 